# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression 
from sklearn.dummy import DummyClassifier
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from scipy import stats as st

### датасет 1

In [3]:
geo_data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
geo_data_0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [5]:
geo_data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


при аназе первого датасета для подготовки к обучению необходимо сделать следующее:\
1. Удалить столбец 'id', поскольку он не является фактором влияния на таргет, а является индетификатором
2. выделить столбец 'product', как таргет.
3. разделить на обучающую и валидационную выборки.  
4. провести масштабирование данных 

In [6]:
target_0 = geo_data_0['product']
geo_0 = geo_data_0.drop(columns =['product', 'id'])

In [7]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    geo_0, target_0, test_size=0.25, random_state=12345)

In [8]:
print('Размер обучающей выборки', features_train_0.shape[0], f'{  features_train_0.shape[0]/geo_0.shape[0]:.1%}')
print('Размер  валидационной выборки', features_valid_0.shape[0], f'{  features_valid_0.shape[0]/geo_0.shape[0]:.1%}')

Размер обучающей выборки 75000 75.0%
Размер  валидационной выборки 25000 25.0%


In [9]:
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(features_train_0.loc[:, numeric])
features_train_0 = scaler.transform(features_train_0.loc[:, numeric])
features_valid_0 = scaler.transform(features_valid_0.loc[:, numeric])

### Датасет 2

In [10]:
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [11]:
geo_data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [12]:
geo_data_1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [13]:
geo_data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


Действия аналогичны, как с первым датасетом

In [14]:
target_1 = geo_data_1['product']
geo_1 = geo_data_1.drop(columns =['product', 'id'])

In [15]:
#scaler.fit(geo_1.loc[:, numeric])
#geo_1.loc[:, numeric] = scaler.transform(geo_1.loc[:, numeric])

In [16]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    geo_1, target_1, test_size=0.25, random_state=12345)

In [17]:
scaler = StandardScaler()
scaler.fit(features_train_1.loc[:, numeric])
features_train_1 = scaler.transform(features_train_1.loc[:, numeric])
features_valid_1 = scaler.transform(features_valid_1.loc[:, numeric])

In [18]:
print('Размер обучающей выборки', features_train_1.shape[0], f'{  features_train_1.shape[0]/geo_1.shape[0]:.1%}')
print('Размер  валидационной выборки', features_valid_1.shape[0], f'{  features_valid_1.shape[0]/geo_1.shape[0]:.1%}')

Размер обучающей выборки 75000 75.0%
Размер  валидационной выборки 25000 25.0%


### Датасет 3

In [19]:
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [20]:
geo_data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [21]:
geo_data_2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


In [22]:
geo_data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [23]:
target_2 = geo_data_2['product']
geo_2 = geo_data_2.drop(columns =['product', 'id'])

In [24]:
#scaler.fit(geo_2.loc[:, numeric])
#geo_2.loc[:, numeric] = scaler.transform(geo_2.loc[:, numeric])

In [25]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    geo_2, target_2, test_size=0.25, random_state=12345)

In [26]:
scaler = StandardScaler()
scaler.fit(features_train_2.loc[:, numeric])
features_train_2 = scaler.transform(features_train_2.loc[:, numeric])
features_valid_2 = scaler.transform(features_valid_2.loc[:, numeric])

In [27]:
print('Размер обучающей выборки', features_train_2.shape[0], f'{  features_train_2.shape[0]/geo_2.shape[0]:.1%}')
print('Размер  валидационной выборки', features_valid_2.shape[0], f'{  features_valid_2.shape[0]/geo_2.shape[0]:.1%}')

Размер обучающей выборки 75000 75.0%
Размер  валидационной выборки 25000 25.0%


в ходе подготовки данных был удален столбец 'id' из всех датасетов\
проведено масштабирование влияющих признаков\
на этапе подготовки получены следующие датасеты: \
для 1 региона - features_train_0, features_valid_0, target_train_0, target_valid_0\
для 2 региона - features_train_1, features_valid_1, target_train_1, target_valid_1\
для 3 региона - features_train_2, features_valid_2, target_train_2, target_valid_2

## Обучение и проверка модели

In [28]:
model_0 = LinearRegression() 
model_0.fit(features_train_0, target_train_0) 
predictions_valid_0 = model_0.predict(features_valid_0)

In [29]:
mse_0 = mean_squared_error(target_valid_0, predictions_valid_0)
rmse_0 = mse_0 **0.5

In [30]:
print('средний запас фактического сырья для первого региона = ', target_valid_0.mean())
print('средний запас предсказанного сырья для первого региона = ', predictions_valid_0.mean())

средний запас фактического сырья для первого региона =  92.07859674082927
средний запас предсказанного сырья для первого региона =  92.59256778438035


In [31]:
print('RMSE = ', rmse_0)

RMSE =  37.5794217150813


In [32]:
model_1 = LinearRegression() 
model_1.fit(features_train_1, target_train_1) 
predictions_valid_1 = model_1.predict(features_valid_1)
mse_1 = mean_squared_error(target_valid_1, predictions_valid_1)
rmse_1 = mse_1 **0.5
print('средний запас фактического сырья для второго региона = ', target_valid_1.mean())
print('средний запас предсказанного сырья для второго региона = ', predictions_valid_1.mean())
print('RMSE = ', rmse_1)

средний запас фактического сырья для второго региона =  68.72313602435997
средний запас предсказанного сырья для второго региона =  68.728546895446
RMSE =  0.893099286775617


In [33]:
model_2 = LinearRegression() 
model_2.fit(features_train_2, target_train_2) 
predictions_valid_2 = model_2.predict(features_valid_2)
mse_2 = mean_squared_error(target_valid_2, predictions_valid_2)
rmse_2 = mse_2 **0.5
print('средний запас фактического сырья для третьего региона = ', target_valid_2.mean())
print('средний запас предсказанного сырья для третьего региона = ', predictions_valid_2.mean())
print('RMSE = ', rmse_2)

средний запас фактического сырья для третьего региона =  94.88423280885438
средний запас предсказанного сырья для третьего региона =  94.96504596800489
RMSE =  40.02970873393434


В ходе обучения моделей наименьшее отклонение по второму региону\
наибольшее количество предсказанного и фактического среднего запаса у третьего региона.\
Среднее предсказанные и фактические запасы сырья приблизительно равны у всех регионов

## Подготовка к расчёту прибыли

In [34]:
deposit = 500
best_deposit = 200
priсe = 450 * 10**3
total = 10 * 10**9
bep = total / (priсe * best_deposit)

In [35]:
print('достаточный объем сырья для безубыточной разработки новой скважины', bep)
print('средний запас фактического сырья для первого региона = ', f'{target_valid_0.mean():.3}', 'тыс. баррелей')
print('средний запас фактического сырья для второго региона = ', f'{target_valid_1.mean():.3}', 'тыс. баррелей')
print('средний запас фактического сырья для третьего региона = ', f'{target_valid_2.mean():.3}', 'тыс. баррелей')

достаточный объем сырья для безубыточной разработки новой скважины 111.11111111111111
средний запас фактического сырья для первого региона =  92.1 тыс. баррелей
средний запас фактического сырья для второго региона =  68.7 тыс. баррелей
средний запас фактического сырья для третьего региона =  94.9 тыс. баррелей


In [36]:
def profit(predictions, target):
    max_predict = pd.Series(predictions).sort_values(
        ascending = False).head(best_deposit)#.reset_index(drop=True)
    target_m = target.reset_index(drop=True)[max_predict.index]#.sort_values(ascending = False).reset_index(drop=True).head(best_deposit)
    a = (target_m.sum()*priсe) - total
    return (a)
   

In [37]:
print('Прибыль для лучших 200 скважен по первому региону = ',
      f'{ (profit(predictions_valid_0, target_valid_0) / 10**9):.3}', 'млрд')
print('Прибыль для лучших 200 скважен по второму региону = ',
      f'{ (profit(predictions_valid_1, target_valid_1) / 10**9):.3}', 'млрд')
print('Прибыль для лучших 200 скважен по третьему региону = ',
      f'{ (profit(predictions_valid_2, target_valid_2) / 10**9):.3}', 'млрд')
      

Прибыль для лучших 200 скважен по первому региону =  3.32 млрд
Прибыль для лучших 200 скважен по второму региону =  2.42 млрд
Прибыль для лучших 200 скважен по третьему региону =  2.71 млрд


Наиболее перспективным можно выделить первый регион

## Расчёт прибыли и рисков 

In [38]:
state = np.random.RandomState(12345)
def prof_risk(predictions, target):
    values = []
    risk = 0
    for i in range(1000):
        #predictions = pd.Series(predictions)
        target = target.reset_index(drop=True)
        subsample_t = target.sample(n=500, replace=True, random_state=state)#.reset_index(drop=True)
        subsample_p = predictions[subsample_t.index]
        values.append(profit(subsample_p, subsample_t))
    for i in values:
        if i < 0:
            risk += i
       
    values = pd.Series(values)
    #confidence_interval = st.t.interval(0.95, df = (len(values) - 1), 
    #                loc = values.mean(), scale = values.sem())
    risk = risk/sum(values)*100    
    mean = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    risk2 = (values < 0).mean()

    print('средняя прибыль = ', f' {mean/10**6:.5}', 'млн')
    print('Риски = ', risk, risk2)
    print('доверительный интервал от ',f'{lower/10**6:.5}', 'млн до', f'{upper/10**6:.5}', 'млн')

In [39]:
prof_risk(predictions_valid_0, target_valid_0)

средняя прибыль =   396.16 млн
Риски =  -1.9346917028984876 0.069
доверительный интервал от  -111.22 млн до 909.77 млн


Для первого региона риски составляют менее 2 %, средняя прибыль варьируется от 111 млн убытков до 900 млн прибыли

In [40]:
prof_risk(predictions_valid_1, target_valid_1)

средняя прибыль =   461.16 млн
Риски =  -0.0689085358593326 0.007
доверительный интервал от  78.051 млн до 862.95 млн


Для второго региона определены незначительные риски, средняя прибыль варьируется от 78 млн до 860 млн

In [41]:
prof_risk(predictions_valid_2, target_valid_2)

средняя прибыль =   392.95 млн
Риски =  -1.7405871695430963 0.065
доверительный интервал от  -112.23 млн до 934.56 млн


Для третьего региона региона риски составляют менее 2 %, средняя прибыль варьируется от 112 млн убытков млрд до 930 млн прибыли

## Общий вывод

Для разработки рекомендуется второй регион, поскольку у данного региона средняя прибыль выше(средняя прибыль составит 461.16 млн), и практически отсутствуют  риски, но максимально возможная прибыль ниже, чем у других регионов, если учесть максимальную прибыль, но с большими рисками, то рекомендуется третий регион. 